# Preparing workspace

In [ ]:
#load "Paket.fsx"
 
Paket.Dependencies.Install """
framework: netstandard2.0
generate_load_scripts: true
storage: none
source https://nuget.org/api/v2
nuget CNTK.CPUOnly
nuget Newtonsoft.Json
nuget MathNet.Numerics
nuget MathNet.Numerics.FSharp
"""

In [1]:
#r "netstandard"
#r @"..\bin\Cntk.Core.Managed-2.6.dll"
#load "../Graphviz/.paket/load/main.group.fsx"

In [2]:
open System
open System.IO

Environment.GetEnvironmentVariable("PATH")
|> fun path -> sprintf "%s%c%s" path (Path.PathSeparator) (Path.GetFullPath("bin"))
|> fun path -> Environment.SetEnvironmentVariable("PATH", path)

open CNTK
DeviceDescriptor.UseDefaultDevice().Type
|> printfn "Congratulations, you are using CNTK for: %A" 

Congratulations, you are using CNTK for: CPU


In [49]:
let device = CNTK.DeviceDescriptor.CPUDevice
let dataType = CNTK.DataType.Float
let initialization = CNTKLib.GlorotUniformInitializer(1.0)
let input_dim, num_output_classes = 2,2

#load "../fsx/CntkHelpers.fsx"
open CntkHelpers

let input = Variable.InputVariable(shape [|input_dim|], dataType, "Features")
let label = Variable.InputVariable(shape [|num_output_classes|], dataType, "Labels")
let z = 
    fullyConnectedClassifierNet input [5] num_output_classes CNTKLib.Sigmoid
    |> (Var>>CNTKLib.Softmax)

In [4]:
let decomposeFunction (root: Function) = 
    let visited = System.Collections.Generic.Dictionary<string, Function>()            

    let rec expand (f: Function) = 
        match visited.ContainsKey(f.Uid) with
        | true -> Seq.empty
        | false -> 
            visited.Add(f.Uid, f)
            seq {
                yield f
                yield! 
                    seq { yield! f.Inputs
                          yield! f.Outputs }
                    |> Seq.map (fun v -> v.Owner)      
                    |> Seq.filter (not<<isNull)
                    |> Seq.collect expand
            }        

    Array.ofSeq (expand root)

In [5]:
open System.Collections.Generic

let (|IsEnumerable|IsDescribable|IsPrimitive|) (t: Type) = 
    if  t <> typeof<string> && 
        (typeof< IEnumerable<_> >).IsAssignableFrom(t) 
    then IsEnumerable
    else if t.GetMethods() 
            |> Array.exists (fun meth -> meth.Name = "AsString") 
    then IsDescribable
    else IsPrimitive

let asString item =
    if  item.GetType().GetMethods() 
        |> Array.exists (fun meth -> meth.Name = "AsString")
    then
        item.GetType()
            .GetMethod("AsString", Array.empty)
            .Invoke(item, Array.empty).ToString()
    else item.ToString()

In [6]:
open System.Reflection

let describeNode (item: obj) =
    [|
        yield KeyValuePair("NodeType", item.GetType().Name)
        yield KeyValuePair("AsString", item |> asString)
        yield!
            item.GetType().GetProperties()
            |> Seq.map
                (fun prop ->
                    match prop.PropertyType with
                    | IsEnumerable -> 
                        prop.Name, 
                          (prop.GetValue(item) :?> IEnumerable<_>) 
                          |> function 
                          | list when list |> Seq.isEmpty -> "[]" 
                          | list -> 
                              list 
                              |> Seq.map (asString)                         
                              |> Seq.reduce (sprintf "%s, %s")
                    | IsDescribable -> prop.Name,asString item
                    | IsPrimitive -> 
                        prop.Name,
                            try prop.GetValue(item) |> asString
                            with ex -> sprintf "%s" ex.Message)
            |> Seq.map (fun (k,v) -> KeyValuePair(k,v))
    |]

In [52]:
// https://github.com/Microsoft/CNTK/blob/master/bindings/python/cntk/logging/graph.py

let extractGraphVizDotNotation (f: Function) = 
        let varText (v:Variable) = (if String.IsNullOrEmpty v.Name then v.Uid else v.Name)// + "\\n" + v.Shape.AsString()
        let funText (f: Function) = if String.IsNullOrEmpty f.Name then f.Uid else f.Name
    
        let varLabel (v: Variable) = sprintf "%s [label=\"%s\"];" v.Uid (varText v)
        let funLabel (f: Function) = sprintf "%s [label=\"%s\"];" f.Uid (funText f)

        let varShape (v: Variable) =
            match v with
            | _ when v.IsInput -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsOutput -> sprintf "%s [shape=invhouse, color=gray];" v.Uid
            | _ when v.IsPlaceholder -> sprintf "%s [shape=invhouse, color=yellow];" v.Uid
            | _ when v.IsParameter -> sprintf "%s [shape=diamond, color=green];" v.Uid
            | _ when v.IsConstant -> sprintf "%s [shape=rectangle, color=lightblue];" v.Uid
            | _ -> sprintf "%s [shape=circle, color=purple];" v.Uid

        let funShape (f: Function) = 
            match f with 
            | _ when f.IsComposite -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, peripheries=2];" f.Uid
            | _ when f.IsPrimitive -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=2, size=0.6];" f.Uid
            | _ -> sprintf "%s [shape=ellipse, fontsize=20, penwidth=4];" f.Uid

        let varEdges (f: Function) (v: Variable) = 
            let inputIndex = f.Inputs |> Seq.map (fun v -> v.Uid) |> Set
            let outputIndex = f.Outputs |> Seq.map (fun v -> v.Uid) |> Set
            match inputIndex.Contains(v.Uid), outputIndex.Contains(v.Uid) with 
            | true, _ when v.IsParameter -> sprintf "%s -> %s [label=\"input param\"];" v.Uid f.Uid |> Some
            | _, true when v.IsParameter -> sprintf "%s -> %s [label=\"output param\"];" f.Uid v.Uid|> Some
            | true, _ -> sprintf "%s -> %s [label=input];" v.Uid f.Uid|> Some 
            | _, true -> 
                if f.Outputs.Count = 1 && f.Output.Uid = v.Uid then
                    //printfn "%s %s %s %s" f.Outputs.[0].Uid f.Output.Uid v.Uid v.Owner.Output.Uid
                    sprintf "%s -> %s [label=output];" f.Uid v.Owner.Output.Uid |> Some
                else sprintf "%s -> %s [label=output];" f.Uid v.Uid|> Some
            //| _ when v.IsParameter -> sprintf "%s -> %s [label=param];" f.Uid v.Uid|> Some
            | _ -> None //sprintf "%s -> %s;" f.Uid v.Uid

        let vars = Seq.append f.Inputs f.Outputs
        let funs = seq { 
                yield f
                yield f.RootFunction;
                yield! vars |> Seq.map (fun v -> v.Owner) |> Seq.filter (isNull>>not) 
            } 

        seq {        
            if f.Uid <> f.RootFunction.Uid then yield sprintf "%s -> %s [label=\"root function\"];" f.RootFunction.Uid f.Uid
            yield! vars |> Seq.map varShape
            yield! vars |> Seq.map varLabel
            yield! vars |> Seq.map (varEdges f) |> Seq.choose id
            yield! funs |> Seq.map funLabel 
            yield! funs |> Seq.map funShape
        } |> Seq.distinct


In [ ]:
let createGraphWithInfo z = ()

In [53]:
createGraphWithInfo z

<div style="width: 100%;min-height: 600px; border: solid lightblue 1px; border-radius: 15px; position: relative">
    <div id="graph" style="width: 100%; height: 100%;"></div>
    <div id="info" style="width: 350px; max-width: 350px; position: absolute; top: 0; right: 0; background-color: white"></div>
    <div style="display: flex;position: absolute; top: 10px; left: 10px; align-items: center;">
        <span>Graphviz engine:</span>
        <select id="engineSelect" class="form-control" style="width: 150px">
            <option>dot</option>
            <option>circo</option>
            <option>fdp</option>
            <option>neato</option>
            <option>osage</option>
            <option>patchwork</option>
            <option>twopi</option>
        </select>
    <div>
</div>

In [22]:
@"<script>
    $('#engineSelect').change(e=> alert($('#engineSelect option:selected').text()))
</script>" |> Util.Html |> Display

In [23]:
@"<script src='../../../d3-jupyter/dist/bundle.js'></script>" |> Util.Html |> Display

In [24]:
let initGraph infoPath graphPath = 
    sprintf "<script>$(document).trigger('INIT_D3', ['%s','%s']);</script>" infoPath graphPath
    |> Util.Html
    
let renderDot dotNotation jsonInfo = 
    sprintf "<script>$(document).trigger('RENDER_GRAPH', [`%s`,`%s`, $('#engineSelect option:selected').text()]);</script>" dotNotation jsonInfo
    |> Util.Html
    
let renderSeries (digraphs : string[]) = 
    digraphs
    |> Array.reduce(sprintf "%s','%s")
    |> sprintf "<script>$(document).trigger('RENDER_SERIES', [['%s']]);</script>"
    |> Util.Html   
    
// """<script>$(document).on("NODE_CLICKED", 
//     (e,d,i,n)=>{
//         console.log(e,d,i,n);
// });</script>""" |> Util.Html
 

In [48]:
open Newtonsoft.Json

let createGraphWithInfo (z: Function) =
    let funcs = z |> decomposeFunction |> Array.filter (fun f -> not f.IsComposite)
    let vars = 
        funcs 
        |> Array.collect(fun f -> [|yield! f.Inputs; yield! f.Outputs|])
        |> Array.distinctBy (fun v -> v.Uid)
    
    let nodeInfo = 
        [|  yield "{"
            yield
                funcs |> Array.map 
                    (fun f -> sprintf "\"%s\": %s" (f.Uid) (f |> describeNode |> JsonConvert.SerializeObject))                            
                |> Array.reduce(sprintf "%s,\n%s")
            yield ","
            yield
                vars |> Array.map 
                    (fun v -> sprintf "\"%s\": %s" (v.Uid) (v |> describeNode |> JsonConvert.SerializeObject))                            
                |> Array.reduce(sprintf "%s,\n%s")
            yield "}" |]
        |> Array.reduce(sprintf "%s\n%s")
        |> fun text -> text.Trim();
    
    let dotNotation =
        funcs
        |> Array.collect (extractGraphVizDotNotation>>Array.ofSeq)
        |> Array.distinct 
        |> Array.reduce(sprintf "%s\n%s")
        |> sprintf "digraph { %s }"
        
    //nodeInfo, dotNotation
    //printfn "%s" nodeInfo 
    //sprintf "<script>console.log(`%s`)</script>" nodeInfo  |>Util.Html
    initGraph "#info" "#graph" |> Display
    renderDot dotNotation nodeInfo |> Display
    //printfn "<script>$(document).trigger('RENDER_GRAPH', [`%s`,`%s`]);</script>" dotNotation nodeInfo
    
createGraphWithInfo z

In [14]:
type Tensor = 
| F of Function 
| V of Variable
with 
    member x.Uid = match x with F f -> f.Uid | V v -> v.Uid
    member x.AsString = match x with F f -> f.AsString() | V v -> v.AsString()

let decomposeTensor (root: Tensor) = 
        
        let visited = Dictionary<string, Tensor>()            

        let rec search (f: Tensor) = 
            visited.Add(f.Uid, f)
        
            seq {   
                match f with
                | F f' -> 
                    yield! f'.Inputs |> Seq.map V
                    yield! f'.Outputs |> Seq.map V
                    yield  f'.RootFunction |> F 
                | V v' -> 
                    yield f
                    if v'.Owner |> isNull |> not then 
                        yield v'.Owner |> F
            }
            |> Seq.filter (fun f -> visited.ContainsKey(f.Uid) |> not)
            |> Seq.iter search

        search root
    
        visited

In [ ]:
decomposeTensor (F z) 
|> Seq.cast<KeyValuePair<string,Tensor>> 
|> Seq.map (fun t -> 
        t.Value 
        |> function 
        | F f -> f |> nodeInfo 
        | V v -> v |> nodeInfo
        |> JsonConvert.SerializeObject
        |> sprintf "\t\"%s\": %s" t.Value.Uid )
|> Seq.take 3
|> Seq.reduce(sprintf "%s,\n%s")        
|> printf "{ \n%s\n}"


In [ ]:
open Newtonsoft.Json
z |> describeNode |> JsonConvert.SerializeObject

In [ ]:
"""<script>
        var dot = `
  digraph {
    Combine1787 -> CompositeFunction1788 [label="root function"];
Combine1787 [label="Combine1787"];
Combine1787 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1789 -> CompositeFunction1790 [label="root function"];
Combine1789 [label="Combine1789"];
Combine1789 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1791 -> CompositeFunction1792 [label="root function"];
Combine1791 [label="Combine1791"];
Combine1791 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1793 -> CompositeFunction1794 [label="root function"];
Combine1793 [label="Combine1793"];
Combine1793 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1795 -> CompositeFunction1796 [label="root function"];
Combine1795 [label="Combine1795"];
Combine1795 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1797 -> CompositeFunction1798 [label="root function"];
Combine1797 [label="Combine1797"];
Combine1797 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1799 -> CompositeFunction1800 [label="root function"];
Combine1799 [label="Combine1799"];
Combine1799 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1801 -> CompositeFunction1802 [label="root function"];
Combine1801 [label="Combine1801"];
Combine1801 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
Combine1803 -> CompositeFunction1804 [label="root function"];
Combine1803 [label="Combine1803"];
Combine1803 [shape=ellipse, fontsize=20, penwidth=2, size=0.6];
  }`;
        var viz = d3.select("#graph")
            .graphviz()
            //.logEvents(true)
            .on("initEnd", function () {

                //viz.engine("circo");
                viz.renderDot(dot)
                    .on("end", function () {

                        let svg = d3.select("svg");//.attr("width", "1750pt").attr("height","1000pt");

                        let defs = svg.append("defs");

                        let filter = defs
                            .append("filter")
                            .attr("id", "shadow")
                            .attr("x", "-50%")
                            .attr("y", "-50%")
                            .attr("width", "200%")
                            .attr("height", "200%");
                        filter.append("feGaussianBlur")
                            .attr("in", "SourceAlpha")
                            .attr("stdDeviation", 3)
                            .attr("result", "blur");
                        filter
                            .append("feOffset")
                            .attr("in", "blur")
                            .attr("dx", 3)
                            .attr("dy", 3);
                        filter.append("feComponentTransfer")
                            .append("feFuncA").attr("type", "linear")
                            .attr("slope", 0.35);

                        var merge = filter.append("feMerge");
                        merge.append("feMergeNode");
                        merge.append("feMergeNode").attr("in", "SourceGraphic");

                        d3.selectAll(".node ellipse, .node polygon")
                            .style("fill", "white")
                            .on("mouseover", (d, i, n) => d3.select(n[i]).style("filter", "url(#shadow)"))
                            .on("mouseout", (d, i, n) => d3.select(n[i]).style("filter", null));
                    });
            });
    </script>""" |> Util.Html |> Display

In [ ]:
"""<script>
        var canvas = d3.select("#graph")
                        .append("svg")
                        .attr("width", 500)
                        .attr("height", 500);
        var circle = canvas.append("circle")
                        .attr("cx",250)
                        .attr("cy", 250)
                        .attr("r", 50)
                        .attr("fill", "red");
    </script>
""" |> Util.Html

In [ ]:
createGraphVizDiagram z |> Seq.reduce(sprintf "%s\t%s\n")

In [ ]:
let printProperty (item: obj) (prop:System.Reflection.PropertyInfo) = 
    ()

type NodeInfo = {Property: string; Value: string} 

let nodeInfo (tensor: obj) = 
    
    let matchTypeName (prop:System.Reflection.PropertyInfo) = 
        try
            match prop.PropertyType.FullName with        
            | "CNTK.Function" -> 
                (prop.GetValue(tensor) :?> CNTK.Function).Uid |> sprintf "fun %s"
            | "CNTK.Variable" -> 
                (prop.GetValue(tensor) :?> CNTK.Variable).Uid |> sprintf "var %s"
            | (list:string) when list.Contains("[CNTK.Variable") -> 
                prop.GetValue(tensor) :?> CNTK.Variable seq 
                |> Seq.map(fun (v: Variable) -> "var " + v.Uid + " " + v.AsString()) 
                |> Seq.reduce(sprintf "%s, %s")
            | (list:string) when list.Contains("[CNTK.Axis") -> 
                prop.GetValue(tensor) :?> CNTK.Axis seq 
                |> Seq.map(fun (a:Axis) -> a.AsString()) 
                |> Seq.reduce(sprintf "%s, %s")
            | "CNTK.NDShape" -> (prop.GetValue(tensor) :?> CNTK.NDShape).AsString()
            | _ -> prop.GetValue(tensor) |> string
        with exn ->
            sprintf "%s: %s -> %s" prop.Name prop.PropertyType.Name exn.Message
    
    seq {
        yield "Type", tensor.GetType().FullName
        match tensor with 
        | :? CNTK.Function -> 
            yield "AsString()", (tensor :?> Function).AsString()
            yield! typeof<Function>.GetProperties()
            |> Array.map (fun prop -> 
                prop.Name, matchTypeName prop)
        | :? CNTK.Variable -> 
            yield "AsString", (tensor :?> Variable).AsString()
            yield! typeof<Variable>.GetProperties()
            |> Array.map (fun prop -> 
                prop.Name, matchTypeName prop)
        | _ ->  (tensor.GetType().FullName) 
                |> sprintf "Unexpected property %s" 
                |> failwith
    } |> Seq.map (fun (k,v) -> { Property= k; Value= v } )

nodeInfo z.Output |> Seq.iter(printfn "%A")
//nodeInfo decz.["Plus18"].Output |> Util.Table
//decz.["Plus18"].Output |> nodeInfo |> Util.Table

//decz.["Plus18"].AsString()

In [ ]:
input.ToFunction().Uid

In [ ]:
"""<script>$(document).on("NODE_CLICKED", 
    (e,d,i,n)=>{
        console.log(e,d,i,n);
});</script>""" |> Util.Html

In [ ]:
"../../../d3-jupyter/dist/bundle.js" |> System.IO.Path.GetFullPath